## Prepare and read data

In [1]:
import gc
import itertools
from pathlib import Path
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD

In [2]:
pd.set_option("max_columns", 300)
pd.set_option("max_rows", 500)

In [3]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

In [4]:
def feature_cache(reset=False):
    def _feature_cache(func):
        def wrapper(train_df, test_df, *args):
            func_name = func.__name__
            train_feat_path = Path("../feature") / f"train_{func_name}.pkl"
            test_feat_path = Path("../feature") / f"test_{func_name}.pkl"
            # if feature exists, load feature
            if train_feat_path.exists() and test_feat_path.exists() and not reset:
                train_feats = pd.read_pickle(train_feat_path).reset_index(drop=True)
                test_feats = pd.read_pickle(test_feat_path).reset_index(drop=True)
                train_df = pd.concat([train_df, train_feats], axis=1)
                test_df = pd.concat([test_df, test_feats], axis=1)
            # if not exists, make feature and save as pickle
            else:
                before_cols = train_df.columns.tolist()
                train_df, test_df = func(train_df, test_df, *args)
                after_cols = train_df.columns.tolist()
                new_cols = [c for c in after_cols if c not in before_cols]
                train_feats = train_df[new_cols]
                test_feats = test_df[new_cols]
                train_feats.to_pickle(train_feat_path)
                test_feats.to_pickle(test_feat_path)            
            return train_df, test_df
        return wrapper

    return _feature_cache

## Feature Engineering

In [5]:
def get_multi_cat_cols(train_df):
    tmp = train_df.iloc[:1000]
    multi_cols = []
    for c in train_df.columns:
        sep_num = tmp[c].astype(str).fillna("").str.contains(";").sum()
        if sep_num > 10:
            multi_cols.append(c)
    return multi_cols

In [6]:
original_cols = train_df.columns.tolist()

In [7]:
multi_cat_cols = get_multi_cat_cols(train_df)

In [8]:
nume_cols = [
    c for c in list(np.setdiff1d(original_cols, multi_cat_cols))
    if c not in ["Salary", "No"] and "float" in train_df[c].dtype.name
]

In [9]:
cat_cols = [c for c in train_df.columns if c not in multi_cat_cols + nume_cols + ["Salary", "No"]]

In [10]:
len(original_cols), len(cat_cols), len(nume_cols), len(multi_cat_cols)

(128, 65, 40, 21)

### rank feature

In [11]:
def add_rank_feature(df):
    rank_prefix_list = [
        "AssessBenefits",
        "AssessJob",
        "JobContactPriorities",
        "JobEmailPriorities",
        "AdsPriorities",
    ]
    for prefix in tqdm(rank_prefix_list):
        rank_cols = [c for c in df.columns if prefix in c]
        col_pairs = itertools.combinations(rank_cols, 2)
        for col_a, col_b in col_pairs:
            df[f"rank_diff_{prefix}_{col_a}_{col_b}"] = (df[col_a] - df[col_b]) / np.log2(df[[col_a, col_b]].max(axis=1))
    return df

In [12]:
train_df = add_rank_feature(train_df)
test_df = add_rank_feature(test_df)

### Multi-category encoding

In [13]:
for c in tqdm(multi_cat_cols):
    binarizer = MultiLabelBinarizer()
    train_multi_srs = train_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    test_multi_srs = test_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    train_arr = binarizer.fit_transform(train_multi_srs)
    test_arr = binarizer.transform(test_multi_srs)
    feat_cols = [f"ohe_{c}_{val}" for val in binarizer.classes_]
    train_feat_df = pd.DataFrame(train_arr, columns=feat_cols, dtype=np.int8)
    test_feat_df = pd.DataFrame(test_arr, columns=feat_cols, dtype=np.int8)
    all_feat_df = pd.concat([train_feat_df, test_feat_df], axis=0, ignore_index=True)
    train_feat_df[f"sum_answer_{c}"] = (train_df[c].str.count(";") + 1).fillna(-1).astype(np.int8)
    test_feat_df[f"sum_answer_{c}"] = (test_df[c].str.count(";") + 1).fillna(-1).astype(np.int8)
    train_df = pd.concat([train_df, train_feat_df], axis=1)
    test_df = pd.concat([test_df, test_feat_df], axis=1)
    # SVD
    svd = TruncatedSVD(n_components=2, random_state=2020)
    all_svd_feats = pd.DataFrame(svd.fit_transform(all_feat_df), columns=[f"svd_{c}_{ix}" for ix in range(2)])
    train_df = pd.concat([train_df, all_svd_feats.iloc[:len(train_df)]], axis=1)
    test_df = pd.concat([test_df, all_svd_feats.iloc[len(train_df):].reset_index(drop=True)], axis=1)

### Simple-category encoding

In [14]:
for c in cat_cols:
    train_df[c], uniques = pd.factorize(train_df[c], sort=True)
    test_df[c] = uniques.get_indexer(test_df[c])

### Reduce data memory

In [15]:
for c in nume_cols:
    if train_df[c].nunique() > 1000:
        train_df[c] = train_df[c].astype(np.float32)
        test_df[c] = test_df[c].astype(np.float32)
    else:
        train_df[c] = train_df[c].astype(np.float16)
        test_df[c] = test_df[c].astype(np.float16)

In [16]:
for c in cat_cols:
    if train_df[c].max() > 32767:
        train_df[c] = train_df[c].astype(np.int32)
        test_df[c] = test_df[c].astype(np.int32)
    else:
        train_df[c] = train_df[c].astype(np.int16)
        test_df[c] = test_df[c].astype(np.int16)            

In [17]:
train_df.shape, test_df.shape

((33857, 659), (11259, 658))

### Change Category Name

In [18]:
train_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df.columns]
test_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_df.columns]

### Aggregation

In [19]:
def get_importance_cols(use_num=50):
    # basicなモデルのimportanceを読み込み
    importance_df = pd.read_csv("../data/importance/003_importance.csv")
    imp_feats = importance_df["feature"].iloc[:use_num].tolist()
    return imp_feats

In [20]:
imp_feats = get_importance_cols(use_num=50)
imp_cat_cols = [c for c in imp_feats if c in cat_cols or c[:4] == "ohe_"]
imp_nume_cols = [c for c in imp_feats if c in nume_cols or c[:4] == "ohe_"]

In [21]:
len(imp_cat_cols), len(imp_nume_cols)

(32, 15)

In [22]:
all_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [23]:
for cat_col in tqdm(imp_cat_cols):
    for nume_col in imp_nume_cols:
        # one-hotは同じカテゴリの場合がある
        if cat_col == nume_col:
            continue
        all_df[f"agg_mean_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("mean").astype(np.float32)
        all_df[f"agg_std_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("std").astype(np.float32)
        all_df[f"agg_max_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("max").astype(np.float32)
        all_df[f"agg_min_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("min").astype(np.float32)
        all_df[f"diff_agg_mean_{cat_col}_{nume_col}"] = \
            all_df[nume_col] - all_df[f"agg_mean_{cat_col}_{nume_col}"]
        all_df[f"rel_agg_mean_{cat_col}_{nume_col}"] = \
            all_df[nume_col] / (1 + all_df[f"agg_mean_{cat_col}_{nume_col}"])
train_df = all_df.iloc[:len(train_df)].reset_index(drop=True)
test_df = all_df.iloc[len(train_df):].reset_index(drop=True)
del all_df
gc.collect()

58

In [24]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]

In [25]:
folds = KFold(n_splits=5, random_state=2020, shuffle=True)
fold_idx_list = [(trn_idx, val_idx) for trn_idx, val_idx in folds.split(train_df, train_df["Salary"])]

### Target Encoding

In [26]:
@feature_cache(reset=False)
def target_encoding(train_df, test_df):
    te_cols = [c for c in train_df.columns if c in cat_cols]
    for c in tqdm(te_cols):
        new_col = "te_" + c
        train_df[new_col] = 0
        test_df[new_col] = 0
        for trn_idx, val_idx in fold_idx_list:
            mean_val = train_df.loc[trn_idx].groupby(c)["Salary"].mean().astype(np.float32)
            train_df.loc[val_idx, new_col] = train_df.loc[val_idx, c].map(mean_val)
            test_df.loc[:, new_col] += test_df.loc[:, c].map(mean_val) / len(fold_idx_list)
        train_df[new_col] = train_df[new_col].astype(np.float32)
        test_df[new_col] = test_df[new_col].astype(np.float32)
    return train_df, test_df

In [27]:
train_df, test_df = target_encoding(train_df, test_df)

In [28]:
@feature_cache(reset=False)
def multiple_target_encoding(train_df, test_df):
     # multiple target encoding
    multi_te_cols = [c for c in train_df.columns if c in cat_cols or c[:4] == "ohe_"]
    imp_feats = get_importance_cols(use_num=30)
    multi_te_cols = [c for c in multi_te_cols if c in imp_feats]
    combi_multi_te_cols = list(itertools.combinations(multi_te_cols, 2))   

    for col_a, col_b in tqdm(combi_multi_te_cols):
        new_col = "te_" + col_a + "__" + col_b
        train_df[new_col] = 0
        test_df[new_col] = 0
        train_df["tmp"] = train_df[col_a].fillna("").astype(str) + train_df[col_b].fillna("").astype(str)
        test_df["tmp"] = test_df[col_a].fillna("").astype(str) + test_df[col_b].fillna("").astype(str)
        for trn_idx, val_idx in fold_idx_list:
            mean_val = train_df.loc[trn_idx].groupby("tmp")["Salary"].mean().astype(np.float32)
            train_df.loc[val_idx, new_col] = train_df.loc[val_idx, "tmp"].map(mean_val)
            test_df.loc[:, new_col] += test_df.loc[:, "tmp"].map(mean_val) / len(fold_idx_list)
        train_df[new_col] = train_df[new_col].astype(np.float32)
        test_df[new_col] = test_df[new_col].astype(np.float32)
        del train_df["tmp"], test_df["tmp"]
        gc.collect()
    return train_df, test_df

In [29]:
train_df, test_df = multiple_target_encoding(train_df, test_df)

In [30]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]
print(len(use_cols))

3725


## Training

In [31]:
lgb_params = {
            'objective': 'poisson',
            "metric": "rmse",
            "verbosity": -1,
            "boosting": "gbdt",
            'learning_rate': 0.05,
            'num_leaves': 64,
            'min_data_in_leaf': 80, 
            'max_depth': 4,
            "bagging_freq": 5,
            "bagging_fraction": 0.8,
            "lambda_l1": 0.5,
            "lambda_l2": 0.5,
            "feature_fraction": 0.1,
            "seed": 2020,
            "num_threads": -1,
            "max_bins": 30
}

In [32]:
importances = pd.DataFrame()
oof_preds = np.zeros(len(train_df))
models = []

for fold_i, (trn_idx, val_idx) in enumerate(fold_idx_list):
    print(f"Fold {fold_i+1}")
    train_dataset = lgb.Dataset(
        train_df.loc[trn_idx, use_cols],
        label = train_df.loc[trn_idx, "Salary"]
    )
    valid_dataset = lgb.Dataset(
        train_df.loc[val_idx, use_cols],
        label = train_df.loc[val_idx, "Salary"]
    )
    model = lgb.train(
                lgb_params,
                train_dataset,
                10000,
                valid_sets = [train_dataset, valid_dataset],
                verbose_eval=100,
                early_stopping_rounds = 300,
                #feval = eval_f1,
                #callbacks = [log_callback],
    )
    imp_df = pd.DataFrame()
    imp_df['feature'] = use_cols
    imp_df['gain'] = model.feature_importance(importance_type="gain")
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    
    oof_preds[val_idx] = model.predict(train_df.loc[val_idx, use_cols])
    models.append(model)

Fold 1
Training until validation scores don't improve for 300 rounds
[100]	training's rmse: 22706.6	valid_1's rmse: 23001.6
[200]	training's rmse: 20875.2	valid_1's rmse: 21615
[300]	training's rmse: 20125.8	valid_1's rmse: 21183.4
[400]	training's rmse: 19657.2	valid_1's rmse: 20976.7
[500]	training's rmse: 19295.9	valid_1's rmse: 20834.1
[600]	training's rmse: 18985.2	valid_1's rmse: 20743.2
[700]	training's rmse: 18709.6	valid_1's rmse: 20680.3
[800]	training's rmse: 18470.5	valid_1's rmse: 20625.4
[900]	training's rmse: 18230.9	valid_1's rmse: 20576.3
[1000]	training's rmse: 18014.2	valid_1's rmse: 20540.1
[1100]	training's rmse: 17814.7	valid_1's rmse: 20495.6
[1200]	training's rmse: 17626	valid_1's rmse: 20462.8
[1300]	training's rmse: 17452.4	valid_1's rmse: 20447.2
[1400]	training's rmse: 17292.4	valid_1's rmse: 20430.6
[1500]	training's rmse: 17115.2	valid_1's rmse: 20414.5
[1600]	training's rmse: 16958	valid_1's rmse: 20391.5
[1700]	training's rmse: 16787.9	valid_1's rmse: 20

[3500]	training's rmse: 14342.3	valid_1's rmse: 20138
[3600]	training's rmse: 14239.9	valid_1's rmse: 20137.6
[3700]	training's rmse: 14107.3	valid_1's rmse: 20140
Early stopping, best iteration is:
[3479]	training's rmse: 14366.2	valid_1's rmse: 20134.8
Fold 5
Training until validation scores don't improve for 300 rounds
[100]	training's rmse: 22731.8	valid_1's rmse: 23006
[200]	training's rmse: 20905.7	valid_1's rmse: 21379.3
[300]	training's rmse: 20180.9	valid_1's rmse: 20875.9
[400]	training's rmse: 19709.7	valid_1's rmse: 20629.2
[500]	training's rmse: 19347.7	valid_1's rmse: 20465.6
[600]	training's rmse: 19035.1	valid_1's rmse: 20351.2
[700]	training's rmse: 18766.2	valid_1's rmse: 20272.4
[800]	training's rmse: 18531.7	valid_1's rmse: 20219.7
[900]	training's rmse: 18310.8	valid_1's rmse: 20168.4
[1000]	training's rmse: 18109	valid_1's rmse: 20129.4
[1100]	training's rmse: 17920.8	valid_1's rmse: 20091.5
[1200]	training's rmse: 17752.3	valid_1's rmse: 20068.4
[1300]	training's

In [33]:
oof_score = np.sqrt(mean_squared_error(train_df["Salary"], oof_preds))
oof_score

20172.29958056713

In [34]:
importances.groupby("feature")["gain"].mean().sort_values(ascending=False).reset_index().iloc[:300]

,feature,gain
0,te_Country__YearsCodingProf,9.583882e+08
1,te_Country__Age,4.030331e+08
2,te_Country__ohe_DevType_Student,3.876314e+08
3,te_Country__Employment,3.158404e+08
4,te_YearsCodingProf__CurrencySymbol,2.397127e+08
5,te_Country__YearsCoding,1.877555e+08
6,te_Country__ohe_RaceEthnicity_White_or_of_Euro...,1.802466e+08
7,te_Country__ohe_Methodology_Agile,1.726670e+08
8,te_Country__ohe_DevType_Engineering_manager,1.633805e+08
9,te_Country__SalaryType,1.447438e+08


## Prediction

In [35]:
test_pred = np.zeros(len(test_df))

for model in models:
    test_pred += model.predict(test_df[use_cols]) / len(models)

In [36]:
test_pred

array([ 69784.97152196,  91202.0346389 ,  84818.93091746, ...,
        88853.61429746,  83013.7773883 , 117593.14566672])

In [37]:
sub_df = pd.read_csv("../input/submit.csv")

In [38]:
sub_df["Salary"] = test_pred

In [40]:
sub_df.to_csv("../predict/022_rank_feature_20172.csv", index=False)